In [ ]:
import retrieval.cache as cache
import signals.ema as ema
import signals.simple_scalping as simple_scalping

import numpy as np
import pandas as pd
import pandas_ta as ta
import plotly.graph_objects as go

from backtesting import Backtest, Strategy
from tqdm import tqdm

### Overarching Constants

In [95]:
ticker = 'TSLA'
start = '2023-01-01'
end = '2024-01-01'
interval = '1d'

columns_open = 'Open'
columns_high = 'High'
columns_low = 'Low'
columns_close = 'Close'
columns_volume = 'Volume'
columns_rsi = 'RSI'
columns_ema_slow = 'EMA Slow' # Need to change function calls below if change this one
columns_ema_fast = 'EMA Fast' # Need to change function calls below if change this one
columns_atr = 'ATR'
column_signal_ema = 'EMA Signal'
columns_simple_scalp = 'Simple Scalp'

refresh_cache=False
back_candle_length = 7

### Make sure we have some data to work with

In [96]:
if not cache.exist_sql_db(ticker, interval) or refresh_cache:
    # load data from yahoo finance and cache it in sql db
    cache.cache_ticker(ticker, interval, start, end)

# Load data from sql db
df = cache.load_ticker(ticker, interval, start, end)

df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2023-01-03,118.470001,118.800003,104.639999,108.099998,108.099998,231402800
1,2023-01-04,109.110001,114.589996,107.519997,113.639999,113.639999,180389000
2,2023-01-05,110.510002,111.750000,107.160004,110.339996,110.339996,157986300
3,2023-01-06,103.000000,114.389999,101.809998,113.059998,113.059998,220911100
4,2023-01-09,118.959999,123.519997,117.110001,119.769997,119.769997,190284000
...,...,...,...,...,...,...,...
245,2023-12-22,256.760010,258.220001,251.369995,252.539993,252.539993,93249800
246,2023-12-26,254.490005,257.970001,252.910004,256.609985,256.609985,86892400
247,2023-12-27,258.350006,263.339996,257.519989,261.440002,261.440002,106494400
248,2023-12-28,263.660004,265.130005,252.710007,253.179993,253.179993,113619900


### Add TA Columns

In [97]:
df[columns_ema_slow] = ta.ema(df[columns_close], length=50)
df[columns_ema_fast] = ta.ema(df[columns_close], length=30)
df[columns_rsi] = ta.rsi(df[columns_close], length=10)

# Average True Range (or Volatility) - help to define stop loss / take profit distance
df[columns_atr] = ta.atr(df[columns_high], df[columns_low], df[columns_close], length=back_candle_length)

# Bollinger bands
df=df.join(ta.bbands(df[columns_close], length=15, std=1.5))

#df


### EMA signal function

In [98]:
# Needed for progress_apply
tqdm.pandas()

df.reset_index(inplace=True)

# row.name is the index of the row -> 'Date' column
df[column_signal_ema] = df.progress_apply(
  lambda row: ema.two_above_or_below(df, row.name, back_candle_length), 
  axis=1
)

#df

100%|██████████| 250/250 [00:00<00:00, 3383.97it/s]


### Calculate the Simple Scalping Signal

In [99]:
df[columns_simple_scalp] = df.progress_apply(
  lambda row: simple_scalping.simple_signal(df, row.name, back_candle_length), axis=1
)

df[df['Simple Scalp'] != 0].head(200)

100%|██████████| 250/250 [00:00<00:00, 1720.93it/s]


,index,Date,Open,High,Low,Close,Adj Close,Volume,EMA Slow,EMA Fast,RSI,ATR,BBL_15_1.5,BBM_15_1.5,BBU_15_1.5,BBB_15_1.5,BBP_15_1.5,EMA Signal,Simple Scalp
68,68,2023-04-12,190.740005,191.580002,180.309998,180.539993,180.539993,150256300,180.646084,186.942502,40.073962,8.630352,181.631731,190.744667,199.857602,9.555114,-0.059900,2,2
74,74,2023-04-20,166.169998,169.699997,160.559998,162.990005,162.990005,210970800,180.620771,184.726134,27.301254,9.317269,172.810082,186.555999,200.301916,14.736505,-0.357200,2,2
75,75,2023-04-21,164.800003,166.000000,161.320007,165.080002,165.080002,123539000,180.011329,183.458642,30.370215,8.389814,169.128504,184.542666,199.956827,16.705255,-0.131324,2,2
76,76,2023-04-24,164.649994,165.649994,158.610001,162.550003,162.550003,140006600,179.326571,182.109697,28.738441,8.119850,167.016006,181.548665,196.081325,16.009657,-0.153654,2,2
77,77,2023-04-25,159.820007,163.470001,158.750000,160.669998,160.669998,121999300,178.594941,180.726491,27.517709,7.439880,163.824205,179.275332,194.726459,17.237315,-0.102070,2,2
78,78,2023-04-26,160.289993,160.669998,153.139999,153.750000,153.750000,153364100,177.620630,178.986072,23.444772,7.457904,159.515792,176.686665,193.857538,19.436524,-0.167895,2,2
89,89,2023-05-11,168.699997,173.570007,166.789993,172.080002,172.080002,103889900,173.462174,172.370292,53.941649,6.486223,156.579408,164.141332,171.703256,9.213918,1.024911,1,1
93,93,2023-05-17,168.410004,174.500000,167.190002,173.860001,173.860001,125473600,172.768073,171.541525,57.237385,6.786196,159.539428,166.318666,173.097903,8.152106,1.056208,1,1
94,94,2023-05-18,174.220001,177.059998,172.449997,176.889999,176.889999,109520300,172.929717,171.886588,61.071076,6.350957,160.063278,167.431999,174.800720,8.802046,1.141767,1,1
95,95,2023-05-19,177.169998,181.949997,176.309998,180.139999,180.139999,136024200,173.212473,172.419066,64.828906,6.208765,159.852860,168.487332,177.121805,10.249403,1.174776,1,1


### Create Dots for Simple Scalping Signals

In [100]:
def render_short_long_signal_point(row: pd.Series, column_name:str) -> str:
  if row[column_name] == simple_scalping.LONG_SIGNAL:
    return row[columns_low]-1e-3
  elif row[column_name] == simple_scalping.SHORT_SIGNAL:
    return row[columns_high]+1e-3
  
  return np.nan

df['ss_point'] = df.apply(lambda row: render_short_long_signal_point(row, columns_simple_scalp), axis=1)

### Plot the Graph

In [101]:
partition = df

fig = go.Figure(data=[go.Candlestick(x=partition.index,
                open=partition[columns_open],
                high=partition[columns_high],
                low=partition[columns_low],
                close=partition[columns_close]),
                
                go.Scatter(x=partition.index, y=partition['BBL_15_1.5'], 
                           line=dict(color='blue', width=1), 
                           name='BBL'),

                go.Scatter(x=partition.index, y=partition['BBU_15_1.5'],
                            line=dict(color='blue', width=1),
                            name='BBU'),

                go.Scatter(x=partition.index, y=partition[columns_ema_slow],
                            line=dict(color='orange', width=1),
                            name='EMA Slow'),

                go.Scatter(x=partition.index, y=partition[columns_ema_fast],
                            line=dict(color='green', width=1),
                            name='EMA Fast'),

                go.Scatter(x=partition.index, y=partition[columns_rsi],
                            line=dict(color='red', width=1),
                            name='RSI'),
                ])

fig.add_trace(go.Scatter(x=partition.index, y=partition['ss_point'],
                            mode='markers', marker=dict(color='darkorange', size=5),
                            name=columns_simple_scalp))

### Backtesting

In [104]:
def signal():
    return df[columns_simple_scalp]

class MyStrategy(Strategy):
    # Stop loss coefficient
    sl_coef = 1.1
    tpsl_ratio = 1.5

    def init(self):
        super().init()

        self.simple_scalp_signal = self.I(signal)

    def next(self):
        super().next()

        # Stop-loss distance
        sl_atr = self.sl_coef * self.data.ATR[-1]
        tpsl_ratio = self.tpsl_ratio

        
        if self.simple_scalp_signal == simple_scalping.LONG_SIGNAL and len(self.trades) == 0:
            sl1 = self.data[columns_close][-1] - sl_atr
            tp1 = self.data[columns_close][-1] + sl_atr * tpsl_ratio
            self.buy(sl=sl1, tp=tp1)

        if self.simple_scalp_signal == simple_scalping.SHORT_SIGNAL and len(self.trades) == 0:
            sl1 = self.data[columns_close][-1] + sl_atr
            tp1 = self.data[columns_close][-1] - sl_atr * tpsl_ratio
            self.sell(sl=sl1, tp=tp1)

bt = Backtest(df, MyStrategy, cash=16000)

bt.run()

/var/folders/_w/4mgyjmws5qq_yf18_4s5kff80000gn/T/ipykernel_36798/3229494173.py:32: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Start                                     0.0
End                                     249.0
Duration                                249.0
Exposure Time [%]                        28.4
Equity Final [$]                  9867.324701
Equity Peak [$]                  16354.958832
Return [%]                         -38.329221
Buy & Hold Return [%]              129.861239
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -39.667689
Avg. Drawdown [%]                   -20.07468
Max. Drawdown Duration                  178.0
Avg. Drawdown Duration                   90.0
# Trades                                 14.0
Win Rate [%]                        14.285714
Best Trade [%]                       7.724013
Worst Trade [%]                     -9.176874
Avg. Trade [%]                    

### Plot the Backtesting Results

In [105]:
bt.plot()

/opt/homebrew/Caskroom/miniforge/base/envs/pytrading/lib/python3.12/site-packages/backtesting/_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.active_drag' property; using the latest value

/opt/homebrew/Caskroom/miniforge/base/envs/pytrading/lib/python3.12/site-packages/backtesting/_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.active_scroll' property; using the latest value



GridPlot(id='p2024', ...)